## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-13-03-29 +0000,nyt,Live Updates: Trump Says U.S. Will ‘Rebuild’ V...,https://www.nytimes.com/live/2026/01/08/world/...
1,2026-01-08-13-03-01 +0000,nyt,Saudi Official Accuses U.A.E. of Helping Yemen...,https://www.nytimes.com/2026/01/08/world/middl...
2,2026-01-08-13-02-56 +0000,bbc,Government to U-turn over business rate rise f...,https://www.bbc.com/news/articles/c8e57dexly1o...
3,2026-01-08-13-00-44 +0000,wapo,"He spends 12 hours making art in the snow, the...",https://www.washingtonpost.com/lifestyle/2026/...
4,2026-01-08-13-00-00 +0000,wsj,The U.S. Pumps More Crude Than Anyone Else. He...,https://www.wsj.com/business/energy-oil/us-ven...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,77
5,venezuela,34
38,oil,28
170,new,24
113,shooting,23


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
379,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,171
197,2026-01-08-00-40-05 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,166
5,2026-01-08-13-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...,162
14,2026-01-08-12-37-40 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,161
39,2026-01-08-11-54-03 +0000,nypost,President Trump’s emotional reaction to watchi...,https://nypost.com/2026/01/08/us-news/presiden...,155


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
379,171,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...
136,95,2026-01-08-06-05-45 +0000,latimes,'Public execution.' 'Trigger happy.' Californi...,https://www.latimes.com/california/story/2026-...
189,70,2026-01-08-01-34-00 +0000,wsj,Trump Administration Draws Up New Legal Justif...,https://www.wsj.com/politics/policy/trump-admi...
5,57,2026-01-08-13-00-00 +0000,wsj,The U.S. pumps more oil than any other country...,https://www.wsj.com/business/energy-oil/us-ven...
21,46,2026-01-08-12-13-22 +0000,nypost,Gov. Hochul announces lofty socialist plan to ...,https://nypost.com/2026/01/08/us-news/gov-hoch...
395,45,2026-01-07-13-34-42 +0000,cbc,U.S. Secretary of State Rubio says he will mee...,https://www.cbc.ca/news/world/us-greenland-eur...
365,43,2026-01-07-16-22-02 +0000,bbc,Canada PM Mark Carney to visit China next week...,https://www.bbc.com/news/articles/cevnek9k32no...
148,42,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
79,39,2026-01-08-10-30-00 +0000,wsj,Some mail won’t be postmarked the day you drop...,https://www.wsj.com/politics/policy/a-letter-w...
394,34,2026-01-07-13-45-59 +0000,cbc,U.S. seeks to assert its control over Venezuel...,https://www.cbc.ca/news/world/united-states-ru...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
